# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [15]:
#r "nuget: Microsoft.SemanticKernel, 0.8.48.1-preview"
#!import config/Settings.cs

using Microsoft.Extensions;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.KernelExtensions;

using System.IO;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.8.48.1-preview

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills and requires Orchestration

In [16]:
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Orchestration.Extensions;

// Load native skill into the kernel registry, sharing its functions with prompt templates
var planner = kernel.ImportSkill(new PlannerSkill(kernel));

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [32]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "WriterSkill");

Define your ASK. What do you want the Kernel to do?

In [33]:
var ask = "I need to email a friend with a list of top 5 places to visit in London.";
var originalPlan = await kernel.RunAsync(ask, planner["CreatePlan"]);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan.Variables.ToPlan().PlanString);

Original plan:

<goal>
I need to email a friend with a list of top 5 places to visit in London.
</goal>
<plan>
  <function.WriterSkill.Brainstorm input="Places to visit in London" setContextVariable="IDEAS"/>
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$IDEAS" bucketCount="5" bucketLabelPrefix="PLACE"/>
  <function.WriterSkill.EmailTo to="friend@example.com" sender="me@example.com">
    <input>
      Hi friend,

      I hope you are doing well and looking forward to your trip to London. Here are some places I recommend you to visit:

      - $PLACE_1: A brief description of why this place is worth visiting.
      - $PLACE_2: A brief description of why this place is worth visiting.
      - $PLACE_3: A brief description of why this place is worth visiting.
      - $PLACE_4: A brief description of why this place is worth visiting.
      - $PLACE_5: A brief description of why this place is worth visiting.

      Let me know if you have any questions or need more suggestions. I can'

As you can see in the above plan, the Planner has taken the user's ask and converted it into an XML-based plan detailing how the AI would go about solving this task.

It makes use of the skills that the Kernel has available to it and determines which functions to call in order to fullfill the user's ask.

The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next skill.

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [40]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Winston Churchill.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(skPrompt, "churchill", "ChurchillSkill", maxTokens: 2000, temperature: 0.2, topP: 0.5);

Let's update our ask using this new skill.

In [41]:
var ask = @"I need to email a friend in Churchill's style, with a list of top 5 places to visit in London.";

In [42]:
var newPlan = await kernel.RunAsync(ask, planner["CreatePlan"]);

In [43]:
Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan.Variables.ToPlan().PlanString);

Updated plan:

<goal>
I need to email a friend in Churchill's style, with a list of top 5 places to visit in London.
</goal>
<plan>
  <function.WriterSkill.Brainstorm input="Top 5 places to visit in London" setContextVariable="PLACES"/>
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$PLACES" bucketCount="5" bucketLabelPrefix="PLACE"/>
  <function.WriterSkill.EmailTo to="friend@example.com" sender="me@example.com">
    <input>
      Dear friend,

      I hope this letter finds you in good health and spirits. I am writing to you to share some of my recommendations for your upcoming trip to London, the capital of the British Empire and the cradle of civilization.

      Here are the top 5 places that you must visit in London, according to my humble opinion:

      - $PLACE_1: A brief description of why this place is worth visiting.
      - $PLACE_2: A brief description of why this place is worth visiting.
      - $PLACE_3: A brief description of why this place is worth visiting.
    

### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a skill called `ExecutePlan`.

In [44]:
var executionResults = newPlan;

In [45]:
int step = 1;
int maxSteps = 5;
while (!executionResults.Variables.ToPlan().IsComplete && step < maxSteps)
{
    var results = await kernel.RunAsync(executionResults.Variables, planner["ExecutePlan"]);
    if (results.Variables.ToPlan().IsSuccessful)
    {
        Console.WriteLine($"Step {step} - Execution results:\n");
        Console.WriteLine(results.Variables.ToPlan().PlanString);

        if (results.Variables.ToPlan().IsComplete)
        {
            Console.WriteLine($"Step {step} - COMPLETE!");
            Console.WriteLine(results.Variables.ToPlan().Result);
            break;
        }
    }
    else
    {
        Console.WriteLine($"Step {step} - Execution failed:");
        Console.WriteLine(results.Variables.ToPlan().Result);
        break;
    }
    
    executionResults = results;
    step++;
    Console.WriteLine("");
}

Step 1 - Execution results:

<goal>
I need to email a friend in Churchill's style, with a list of top 5 places to visit in London.
</goal><plan>
  <function._GLOBAL_FUNCTIONS_.BucketOutputs input="$PLACES" bucketCount="5" bucketLabelPrefix="PLACE" />
  <function.WriterSkill.EmailTo to="friend@example.com" sender="me@example.com"><input>
      Dear friend,

      I hope this letter finds you in good health and spirits. I am writing to you to share some of my recommendations for your upcoming trip to London, the capital of the British Empire and the cradle of civilization.

      Here are the top 5 places that you must visit in London, according to my humble opinion:

      - $PLACE_1: A brief description of why this place is worth visiting.
      - $PLACE_2: A brief description of why this place is worth visiting.
      - $PLACE_3: A brief description of why this place is worth visiting.
      - $PLACE_4: A brief description of why this place is worth visiting.
      - $PLACE_5: A brief